In [19]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import matplotlib.pyplot as plt
import scipy.misc
import sklearn.feature_extraction
import numpy as np
import scipy.ndimage
from skimage import measure, io
from skimage import transform
import train_rgb 
import skimage
import json
import datetime
import pickle
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.4
#set_session(tf.Session(config=config))

save_results = r'C:\Users\buggyr\Mosaic_Experiments\models'
epoch_cb_dir = r'C:\Users\buggyr\Mosaic_Experiments\data\interim\Callback_images'
load_training = r'C:\Users\buggyr\Mosaic_Experiments\data\external\D1'

In [20]:
import keras
from keras.models import *
from keras.models import Sequential
from keras.layers import Conv2D, Activation, UpSampling2D,Reshape, Input, Conv2DTranspose, Lambda
from keras.optimizers import Adadelta, Nadam, RMSprop
from keras.models import load_model

In [23]:
f1 = 64
f2 = 32
f3 = 32
ff = 3
k1 = (9,9)
k2 = (1,1)
k3 = (5,5)


inputs = Input(shape = (None, None, 4))

conv1 = Conv2D(f1, (3, 3), activation='relu', padding='same')(inputs)

conv2 = Conv2D(f1, (3, 3), activation='relu', padding='same')(conv1)

conv3 = Conv2D(f1, (3, 3), activation='relu', padding='same')(conv2)

conv4 = Conv2DTranspose(f1, (2, 2), strides=(2, 2), padding='same')(conv3)

conv5 = Conv2D(f1, (3, 3), activation='relu', padding='same')(conv4)

conv6 = Conv2D(f1, (3, 3), activation='relu', padding='same')(conv5)

conv7 = Conv2D(3, (3, 3), activation='relu', padding='same')(conv6)

#out = Lambda(train_rgb.clipper, name = 'clipper')(conv7)

model = Model(inputs=[inputs], outputs=[out])

optimizer_func = Nadam(lr=0.00002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
loss_func='mse'
model.compile(optimizer=optimizer_func,loss=loss_func)

modsum = model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, None, 4)     0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, None, None, 64)    2368      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, None, None, 64)    36928     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, None, None, 64)    36928     
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, None, None, 64)    16448     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, None, None, 64)    36928     
_________________________________________________________________
conv2d_17 (Conv2D)           (None, None, None, 64)    36928     
__________

In [24]:
keyname = "_6layer_Transpose"
now=datetime.datetime.now()
save_file=os.path.join(save_results,now.strftime("%Y-%m-%d %H-%M")+keyname)
os.mkdir(save_file)
save_pred = os.path.join(save_file,'Epoch_Predictions')
os.mkdir(save_pred)
save_model = os.path.join(save_file,'Epoch_Models')
os.mkdir(save_model)
save_test = os.path.join(save_file,'Test_Results')
os.mkdir(save_test)

with open(os.path.join(save_file,'Model_Summary.txt'),'w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
tbCallBack = keras.callbacks.TensorBoard(log_dir=os.path.join(save_file,'TNSR_BRD'), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = keras.callbacks.CSVLogger(os.path.join(save_file,'training.log'), separator=',', append=False)
epoch_predict = train_rgb.Save_predictions(save_pred, epoch_cb_dir)
model_checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(save_model,'model.{epoch:02d}-{loss:.2f}.hdf5'), monitor='loss')

In [25]:
fls = len(os.listdir(load_training))

                            #train_dir, patch_size, batch_size
train_generator = train_rgb.train_generator_rgb(load_training,64,32)
history = model.fit_generator(generator = train_generator,steps_per_epoch=3*fls,
                              verbose = 1, epochs = 200,callbacks = [tbCallBack,csv_logger,
                                epoch_predict, model_checkpoint])
print(history.history)

model.save(os.path.join(save_file,'DeMos_mod.h5'))

Epoch 1/200
5/6 [========================>.....] - ETA: 0s - loss: 0.4585C:\Users\buggyr\Mosaic_Experiments\data\interim\Callback_images\1.tif


C:\ProgramData\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: C:\Users\buggyr\Mosaic_Experiments\models\2017-12-28 21-36_6layer_Transpose\Epoch_Predictions\0_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\ProgramData\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: C:\Users\buggyr\Mosaic_Experiments\models\2017-12-28 21-36_6layer_Transpose\Epoch_Predictions\0_7.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\ProgramData\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: C:\Users\buggyr\Mosaic_Experiments\models\2017-12-28 21-36_6layer_Transpose\Epoch_Predictions\0_kodim01.png is a low contrast image
  warn('%s is a low contrast image' % fname)


C:\Users\buggyr\Mosaic_Experiments\data\interim\Callback_images\7.tif
C:\Users\buggyr\Mosaic_Experiments\data\interim\Callback_images\kodim01.png
C:\Users\buggyr\Mosaic_Experiments\data\interim\Callback_images\kodim08.png
C:\Users\buggyr\Mosaic_Experiments\data\interim\Callback_images\kodim19.png


C:\ProgramData\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: C:\Users\buggyr\Mosaic_Experiments\models\2017-12-28 21-36_6layer_Transpose\Epoch_Predictions\0_kodim08.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\ProgramData\Anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: C:\Users\buggyr\Mosaic_Experiments\models\2017-12-28 21-36_6layer_Transpose\Epoch_Predictions\0_kodim19.png is a low contrast image
  warn('%s is a low contrast image' % fname)


UnicodeDecodeError: 'rawunicodeescape' codec can't decode bytes in position 84-85: truncated \UXXXXXXXX escape

In [ ]:
ptch_sizes = [64, 128]
for ptch_size in ptch_sizes:
    print(str(ptch_size))
    
    now=datetime.datetime.now()
    save_file=os.path.join(save_results,now.strftime("%Y-%m-%d %H-%M"))
    os.mkdir(save_file)
    save_pred = os.path.join(save_file,'Epoch_Predictions')
    os.mkdir(save_pred)

    with open(os.path.join(save_file,'Model_Summary.txt'),'w') as fh:
        # Pass the file handle in as a lambda function to make it callable
        model.summary(print_fn=lambda x: fh.write(x + '\n'))

    #rmsprop = RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0)
    optimizer_func = Nadam(lr=0.00002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    loss_func='mse'

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
    tbCallBack = keras.callbacks.TensorBoard(log_dir=os.path.join(save_file,'TNSR_BRD'), histogram_freq=0, write_graph=True, write_images=True)
    csv_logger = keras.callbacks.CSVLogger(os.path.join(save_file,'training.log'), separator=',', append=False)
    epoch_predict = train_rgb.Save_predictions(save_pred)


    model.compile(optimizer=optimizer_func,loss=loss_func)

    fls = len(os.listdir(load_training))

                                #train_dir, patch_size, batch_size
    train_generator = train_rgb.train_generator_rgb(load_training,ptch_size,32)
    history = model.fit_generator(generator = train_generator,steps_per_epoch=3*fls, verbose = 1,
                                 epochs = 200,callbacks = [tbCallBack,csv_logger,epoch_predict])
    print(history.history)

    model.save(os.path.join(save_file,'DeMos_mod.h5'))
    
    #Test Kodak
    data ={}

    kodak_dir = r'C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak'
    ls = len(os.listdir(kodak_dir))

    kodak_generator = train_rgb.predict_generator_rgb(kodak_dir)

    # k_pred = model.predict_generator(kodak_generator, steps = ls)

    res_Kodak = train_rgb.predict_generator(model,kodak_generator,ls,data)

    data['Kodak_IMGS_PSNR'] = res_Kodak[0]
    data['Kodak_IMGS_SSIM'] = res_Kodak[1]
    data['Kodak_AVG_PSNR']  = res_Kodak[2]
    data['Kodak_AVG_SSIM']  = res_Kodak[3]
    
    #Test McManus
    McM_dir = r'C:\Users\buggyr\Mosaic_Experiments\data\interim\McM'
    ls = len(os.listdir(McM_dir))

    McM_generator = train_rgb.predict_generator_rgb(McM_dir)

    # k_pred = model.predict_generator(kodak_generator, steps = ls)

    res_McM = train_rgb.predict_generator(model,McM_generator,ls,data)

    data['McM_IMGS_PSNR'] = res_McM[0]
    data['McM_IMGS_SSIM'] = res_McM[1]
    data['McM_AVG_PSNR']  = res_McM[2]
    data['McM_AVG_SSIM']  = res_McM[3]
    
    #Write Results
    data['Parameters'] = {
        'Loss Function': loss_func,
        'Optimizer':str(type(optimizer_func))
    }
    data['Training Set'] = {
        'Training Path': load_training,
    }

    with open(os.path.join(save_file,'results.txt'), 'w') as outfile:  
        json.dump(data, outfile,indent=4)

In [27]:
#Test Kodak
data ={}

kodak_dir = r'C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak'
ls = len(os.listdir(kodak_dir))

kodak_generator = train_rgb.predict_generator_rgb(kodak_dir)

# k_pred = model.predict_generator(kodak_generator, steps = ls)

res_Kodak = train_rgb.predict_generator(model,kodak_generator,ls,data,5,save_test)

data['Kodak_IMGS_PSNR'] = res_Kodak[0]
data['Kodak_IMGS_SSIM'] = res_Kodak[1]
data['Kodak_AVG_PSNR']  = res_Kodak[2]
data['Kodak_AVG_SSIM']  = res_Kodak[3]

C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim01.png
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim02.png
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim03.png
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim04.png
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim05.png
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim06.png
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim07.png
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim08.png
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim09.png
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim10.png
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim11.png
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim12.png
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim13.png
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak\kodim14.png
0.0
C:\Users\buggyr\Mosa

In [24]:
#Test McManus
McM_dir = r'C:\Users\buggyr\Mosaic_Experiments\data\interim\McM'
ls = len(os.listdir(McM_dir))

McM_generator = train_rgb.predict_generator_rgb(McM_dir)

# k_pred = model.predict_generator(kodak_generator, steps = ls)

res_McM = train_rgb.predict_generator(model,McM_generator,ls,data,5,save_test)

data['McM_IMGS_PSNR'] = res_McM[0]
data['McM_IMGS_SSIM'] = res_McM[1]
data['McM_AVG_PSNR']  = res_McM[2]
data['McM_AVG_SSIM']  = res_McM[3]

C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\1.tif
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\10.tif
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\11.tif
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\12.tif
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\13.tif
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\14.tif
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\15.tif
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\16.tif
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\17.tif
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\18.tif
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\2.tif
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\3.tif
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\4.tif
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\5.tif
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\6.tif
0.0
C:\Users\buggyr\Mosaic_Experiments\data\interim\McM\7.tif
0.0

In [25]:
#Write Results
data['Parameters'] = {
    'Loss Function': loss_func,
    'Optimizer':str(type(optimizer_func))
}
data['Training Set'] = {
    'Training Path': load_training,
}

with open(os.path.join(save_file,'results.txt'), 'w') as outfile:  
    json.dump(data, outfile,indent=4)

In [17]:
import importlib
importlib.reload(train_rgb)

<module 'train_rgb' from 'C:\\Users\\buggyr\\Mosaic_Experiments\\src\\train_rgb.py'>

In [26]:
model = keras.models.load_model(r'C:\Users\buggyr\Mosaic_Experiments\models\2017-12-27 21-49_6layer_Transpose\DeMos_mod.h5')

In [ ]:
model.save(os.path.join(save_file,'DeMos_mod.h5'))

In [15]:
save_file =r'C:\Users\buggyr\Mosaic_Experiments\models\2017-12-20 19-06_10_layer_Transpose'
save_pred = os.path.join(save_file,'Epoch_Predictions')
tbCallBack = keras.callbacks.TensorBoard(log_dir=os.path.join(save_file,'TNSR_BRD'), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = keras.callbacks.CSVLogger(os.path.join(save_file,'training.log'), separator=',', append=False)
epoch_predict = train_rgb.Save_predictions(save_pred)

In [4]:
save_test = r'C:\Users\buggyr\Mosaic_Experiments\models\2017-12-20 19-06_10_layer_Transpose\Test_Results'